In [ ]:
#@markdown ####Fetch
%cd

!git clone -b dev https://github.com/jzcurious/cuda-sample1.git
%cd cuda-sample1

project_dir = !pwd
project_dir = project_dir[0]


####Debug build (host)

In [ ]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DCMAKE_BUILD_TYPE=DebugHost ..
!cmake --build . -j4
%cd tests

In [ ]:
!ASAN_OPTIONS="protect_shadow_gap=0" ./cuda-sample1-test --gtest_brief=1

####Debug build (device)

In [ ]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DCMAKE_BUILD_TYPE=DebugDevice ..
!cmake --build . -j4
%cd tests

In [ ]:
!compute-sanitizer --target-processes all --tool memcheck --leak-check=full ./cuda-sample1-test --gtest_brief=1

####Release build

In [ ]:
%cd $project_dir
!mkdir -p build
%cd build
!cmake --fresh -DTESTING=1 -DBENCHMARKING=1 -DCMAKE_BUILD_TYPE=Release ..
!cmake --build . -j4
%cd tests

In [ ]:
!ctest

####Benchmarking

#####Linear job growth

In [ ]:
#@markdown $G(x) = 100x + b$
%cd $project_dir/build/benchmarks

b = 1000 # @param {type:"number"}
x = 10000 # @param {type:"number"}

!./dotbench -b $b -n $x -o dot_host.csv
!./dotbench -b $b -n $x -o dot_device.csv -d

if x >= 200:
    %run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (linear job)"
else:
    %run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (linear job)" -m

#####Exponential job growth

In [ ]:
#@markdown $G(x) = 10^x + b$
%cd $project_dir/build/benchmarks

b = 1000 # @param {type:"number"}
x = 10 # @param {type:"number"}

!./dotbench -e -b $b -n $x -o dot_host.csv
!./dotbench -e -b $b -n $x -o dot_device.csv -d

%run vis.py -i dot_host.csv dot_device.csv -l Host Device -c indianred limegreen -t "Real Complexity of Dot Product (exponential job)" -m